In [2]:
!pip install pymysql mysql-connector-python
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import os
import mysql.connector

Defaulting to user installation because normal site-packages is not writeable


In [2]:
username = 'test'
password = '123456789'
host = 'localhost' 
port = '3306'      
database = 'sphinx'

In [20]:
connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()

# Get a list of all tables in the database
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

# Iterate over each table and delete all rows
for (table_name,) in tables:
    try:
        # Delete or truncate the table
        truncate_query = f"TRUNCATE TABLE {table_name};"
        cursor.execute(truncate_query)
        print(f"Table '{table_name}' has been truncated.")
    except Exception as e:
        print(f"Failed to truncate table '{table_name}': {e}")

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

Table 'badges' has been truncated.
Table 'comments' has been truncated.
Table 'linktypes' has been truncated.
Table 'postlinks' has been truncated.
Table 'posts' has been truncated.
Table 'posttypes' has been truncated.
Table 'users' has been truncated.
Table 'votes' has been truncated.
Table 'votetypes' has been truncated.


In [18]:
#Fixing Votes.csv

df = pd.read_csv("../data/Badges.csv", index_col=False)
df = df.iloc[:, ~df.columns.str.contains('^Unnamed')]
df

,Id,Name,UserId,Date
0,82946,Teacher,3718,2008-09-15 08:55:03.923
1,82947,Teacher,994,2008-09-15 08:55:03.957
2,82949,Teacher,3893,2008-09-15 08:55:03.957
3,82950,Teacher,4591,2008-09-15 08:55:03.957
4,82951,Teacher,5196,2008-09-15 08:55:03.957
...,...,...,...,...
444068,9770257,Mortarboard,3535708,2008-10-06 18:56:55.270
444069,9770258,Mortarboard,3540161,2009-02-03 14:12:09.643
444070,13192978,Mortarboard,646723,2009-05-08 06:14:13.743
444071,13431590,Mortarboard,4549416,2008-10-10 23:04:45.833


In [21]:
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')
folder_path = "../data"
for file_name in os.listdir(folder_path):
    # Check if the file is a CSV
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        table_name=(file_name.split(".")[0]).lower()
        print(f"Reading file: {file_name}")
        file_size = os.path.getsize(file_path)  # Size in bytes 
        
        if file_size / (1024 ** 2)>15:
            chunksize = 10000  
            try:
                with engine.connect() as connection:
                    print(f"Dataframe will be inserted in chunks: {file_name}")
                    for chunk in pd.read_csv(file_path, chunksize=chunksize, index_col=False):
                        chunk = chunk.iloc[:, ~chunk.columns.str.contains('^Unnamed')]
                        chunk.to_sql(name=table_name, con=engine, if_exists='append', index=False)
                        
            except Exception as e:
                print(f"An error occurred: {e}")
                # Explicit rollback if the connection supports transactions
                connection.rollback()
            finally:
                engine.dispose()
        else:
            df = pd.read_csv(file_path, index_col=False)
            df = df.iloc[:, ~df.columns.str.contains('^Unnamed')]
            print(f"Dataframe ready for MySQL: {file_name}")
            
            try:
                with engine.connect() as connection:
                    df.to_sql(table_name, con=connection, if_exists='replace', index=False)
            except Exception as e:
                print(f"An error occurred: {e}")
                # Explicit rollback if the connection supports transactions
                connection.rollback()
            finally:
                engine.dispose()
                

Reading file: Badges.csv
Dataframe will be inserted in chunks: Badges.csv
Reading file: Comments.csv
Dataframe will be inserted in chunks: Comments.csv
Reading file: LinkTypes.csv
Dataframe ready for MySQL: LinkTypes.csv
Reading file: PostLinks.csv
Dataframe ready for MySQL: PostLinks.csv
Reading file: Posts.csv
Dataframe will be inserted in chunks: Posts.csv
Reading file: PostTypes.csv
Dataframe ready for MySQL: PostTypes.csv
Reading file: Users.csv
Dataframe will be inserted in chunks: Users.csv
Reading file: Votes.csv
Dataframe will be inserted in chunks: Votes.csv
Reading file: VoteTypes.csv
Dataframe ready for MySQL: VoteTypes.csv


You should install sphinx and put the binaries folder in this folder.

In [3]:
import subprocess

result = subprocess.run(
    ["sphinx-3.7.1\\bin\\indexer", "--config", "sphinx-min.conf.dist", "--all"],
    capture_output=True,
    text=True
)
print(result.stdout) 
print(result.stderr) 


Sphinx 3.7.1-dev (commit da9f8a4e7)
Copyright (c) 2001-2024, Andrew Aksyonoff
Copyright (c) 2008-2016, Sphinx Technologies Inc (http://sphinxsearch.com)

using config file 'sphinx-min.conf.dist'...
indexing index 'comments'...
collected 1373756 docs, 203.7 MB
sorted 37.2 Mhits, 100.0% done
total 1373756 docs, 203.7 Mb
total 28.8 sec, 7.076 Mb/sec, 47723 docs/sec
indexing index 'posts'...
collected 1565425 docs, 1101.4 MB
sorted 183.3 Mhits, 100.0% done
total 1565425 docs, 1.101 Gb
total 138.7 sec, 7.942 Mb/sec, 11287 docs/sec
indexing index 'users'...
collected 99869 docs, 9.2 MB
sorted 1.5 Mhits, 100.0% done
total 99869 docs, 9.171 Mb
total 1.6 sec, 5.775 Mb/sec, 62880 docs/sec
indexing index 'usersjoincomments'...
collected 831529 docs, 215.5 MB
sorted 37.5 Mhits, 100.0% done
total 831529 docs, 215.5 Mb
total 36.4 sec, 5.913 Mb/sec, 22816 docs/sec
indexing index 'usersjoinposts'...
collected 948342 docs, 770.1 MB
sorted 128.6 Mhits, 100.0% done
total 948342 docs, 770.1 Mb
total 107.3